In [ ]:
!git clone https://github.com/visirion07/mpiigaze.git

In [ ]:
import os

import csv
import sys
from spacy.lang.en import English
from io import open
import string
import re
import random
import nltk
import time
import math
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from nltk.tokenize import word_tokenize
import operator
from queue import PriorityQueue
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu, SmoothingFunction
# from nltk.translate.meteor_score import single_meteor_score
from nltk.tokenize.treebank import TreebankWordDetokenizer
import numpy as np

In [ ]:
datag = {}
tot = 0
for f in os.listdir("/content/mpiigaze/outputs"):
  person = int(f.split('.')[0][1:-1])
  if (f.split('.')[0][-1]!='b'):
    # print(f)
    continue
  print(f)
  arr = np.load("/content/mpiigaze/outputs/" + f, allow_pickle = True)
  datag[person] = {}
  tot += arr.shape[0]
  datag[person]["gaze"] = []
  datag[person]["gazep"] = []
  datag[person]["z_app"] = []
  datag[person]["z_gaze"] = []
  datag[person]["head"] = []
  datag[person]["R"] = []
  for op in arr:
    datag[person]["gaze"].append(op[0])
    datag[person]["gazep"].append(op[3])
    datag[person]["z_gaze"].append(op[2].reshape(-1, 6))
    datag[person]["z_app"].append(op[1])
    datag[person]["head"].append(op[-4])
    datag[person]["R"].append(op[-2])
  datag[person]["gaze"] =  np.array(datag[person]["gaze"])
  datag[person]["head"] =  np.array(datag[person]["head"])  
  datag[person]["gazep"] = np.squeeze(np.array(datag[person]["gazep"]), axis = 1)
  datag[person]["z_app"] = np.squeeze(np.array(datag[person]["z_app"]), axis = 1)
  datag[person]["z_gaze"] = np.squeeze(np.array(datag[person]["z_gaze"]), axis = 1)
  datag[person]["R"] = np.array(datag[person]["R"])
  print(person)
  print(datag[person]["gaze"].shape)
  print(datag[person]["gazep"].shape)
  print(datag[person]["z_app"].shape)
  print(datag[person]["z_gaze"].shape)
  print(datag[person]["head"].shape)
  print(datag[person]["R"].shape)
print("Total data ", tot)

p01b.npy
1
(24143, 2)
(24143, 3)
(24143, 64)
(24143, 6)
(24143, 2)
(24143, 3, 3)
p04b.npy
4
(16831, 2)
(16831, 3)
(16831, 64)
(16831, 6)
(16831, 2)
(16831, 3, 3)
p13b.npy
13
(1498, 2)
(1498, 3)
(1498, 64)
(1498, 6)
(1498, 2)
(1498, 3, 3)
p08b.npy
8
(10701, 2)
(10701, 3)
(10701, 64)
(10701, 6)
(10701, 2)
(10701, 3, 3)
p02b.npy
2
(28019, 2)
(28019, 3)
(28019, 64)
(28019, 6)
(28019, 2)
(28019, 3, 3)
p14b.npy
14
(1500, 2)
(1500, 3)
(1500, 64)
(1500, 6)
(1500, 2)
(1500, 3, 3)
p09b.npy
9
(7995, 2)
(7995, 3)
(7995, 64)
(7995, 6)
(7995, 2)
(7995, 3, 3)
p11b.npy
11
(2982, 2)
(2982, 3)
(2982, 64)
(2982, 6)
(2982, 2)
(2982, 3, 3)
p10b.npy
10
(2810, 2)
(2810, 3)
(2810, 64)
(2810, 6)
(2810, 2)
(2810, 3, 3)
p03b.npy
3
(35075, 2)
(35075, 3)
(35075, 64)
(35075, 6)
(35075, 2)
(35075, 3, 3)
p07b.npy
7
(15509, 2)
(15509, 3)
(15509, 64)
(15509, 6)
(15509, 2)
(15509, 3, 3)
p05b.npy
5
(16577, 2)
(16577, 3)
(16577, 64)
(16577, 6)
(16577, 2)
(16577, 3, 3)
p12b.npy
12
(1609, 2)
(1609, 3)
(1609, 64)
(1609, 6)
(

In [ ]:
import math

In [ ]:
def mean_angle_loss(pred, truth):
    '''
    :param pred,truth: type=torch.Tensor
    :return:
    '''
    pred = pred.detach().numpy()
    ans = 0
    for i in range(len(pred)):
        p_x, p_y, p_z = (pred[i][j] for j in range(3))
        t_x, t_y, t_z = (truth[i][j] for j in range(3))
        
        # print("p_x={}, p_y={}, p_z={}".format(p_x, p_y, p_z))
        # print("t_x={}, t_y={}, t_z={}".format(t_x, t_y, t_z))
        angles = (p_x * t_x + p_y * t_y + p_z * t_z)/(math.sqrt(p_x**2+p_y**2+p_z**2) * math.sqrt(t_x**2+t_y**2+t_z**2))
        ans += math.acos(angles) * 180 / np.pi
    return ans / len(pred)
def angular_diff(py1, py2):
  # xzLen = cos(pitch)
  # x = xzLen * cos(yaw)
  # y = sin(pitch)
  # z = xzLen * sin(-yaw)
  xz1 = math.cos(py1[0])
  x1 = xz1*math.cos(py1[1])
  y1 = math.sin(py1[0])
  z1 = xz1 * math.sin(-py1[1])
  y_1 = [x1, y1, z1]

  xz2 = math.cos(py2[0])
  x2 = xz2*math.cos(py2[1])
  y2 = math.sin(py2[0])
  z2 = xz2 * math.sin(-py2[1])
  y_2 = [x2, y2, z2]

  angle_dot = (x1 * x2 + y1*y2 + z1 * z2)/(math.sqrt(x1**2+y1**2+z1**2) * math.sqrt(x2**2+ y2**2+z2**2))
  ans = math.acos(angle_dot) * 180 / np.pi
  return ans

In [ ]:
class Gazeo(nn.Module):
  def __init__(self, input_size):
    super(Gazeo, self).__init__()
    self.fcc1 = nn.Linear(input_size, 64)
    self.fcc2 = nn.Linear(64, 3)
    self.drop = nn.Dropout(0.2)
    self.act = nn.ReLU()
  def forward(self, input, output):
    expand = self.drop(self.fcc1(input))
    expan = self.act(expand)
    out = self.fcc2(expan)
    return out

In [ ]:
class Gazeonly(nn.Module):
  def __init__(self, input_size):
    super(Gazeonly, self).__init__()
    self.fcc1 = nn.Linear(input_size, 64)
    self.fcc2 = nn.Linear(64, 3)
    self.act = nn.ReLU()
  def forward(self, input, output):
    expand = self.fcc1(input)
    expan = self.act(expand)
    out = self.fcc2(expan)
    return out

In [ ]:
class Gaze(nn.Module):
  def __init__(self, input_1_size, input_2_size):
    super(Gaze, self).__init__()
    self.fcc1 = nn.Linear(input_1_size, 24)
    self.fcc2 = nn.Linear(input_2_size, 16)
    self.fcc3 = nn.Linear(input_2_size, 8)
    self.fcc4 = nn.Linear(40, 16)
    self.fcc5 = nn.Linear(16, 8)
    self.fcc6 = nn.Linear(16, 8)
    self.fcc7 = nn.Linear(8, 3)
    self.act = nn.ReLU()
  def forward(self, input1, input2, output):
    # print(input1.shape)
    minimize_1 = self.fcc1(input1)
    min_1 = self.act(minimize_1)
    # print(min_1.shape)
    minimize_2 = self.fcc2(input2)
    min_2 = self.act(minimize_2)
    # print(min_2.shape)
    min_3 = self.fcc3(input2)
    concat_1 = torch.cat((min_1, min_2), 1)
    # print(concat_1.shape)
    minimize_4 = self.fcc4(concat_1)
    min_4 = self.act(minimize_4)
    min_6 = self.fcc6(min_4)
    # min_6 = self.fcc6(min_5)
    output_h = self.fcc7(min_6)
    # print(min_4.shape, min_6.shape, output_h.shape)
    return output_h

In [ ]:
class BigGaze(nn.Module):
  def __init__(self, input_1_size, input_2_size):
    super(BigGaze, self).__init__()
    self.fcc1 = nn.Linear(64, 32)
    self.fcc2 = nn.Linear(32, 16)
    self.fcc3 = nn.Linear(16, 6)
    self.fcc4 = nn.Linear(6, 12)
    self.fcc5 = nn.Linear(18, 3)
    self.act1 = nn.ReLU()
    self.act2 = nn.ReLU()
    self.act3 = nn.ReLU()
    self.act4 = nn.ReLU()

  def forward(self, input1, input2, output):
    # print(input1.shape, input2.shape)
    exp = self.act1(self.fcc1(input1))
    exp_1 =  self.act2(self.fcc2(exp))
    mini =  self.act3(self.fcc3(exp_1))
    mini_1 =  self.act4(self.fcc4(input2))
    comb = torch.cat((mini, mini_1), 1)
    comb1 =  self.fcc5(comb)

    # output_h = self.fcc11(self.fcc10(comb1))
    
    return comb1


In [ ]:
import numpy as np
import torch
import torch.nn.functional as F


def nn_angular_distance(a, b):
    sim = F.cosine_similarity(a, b, eps=1e-6)
    sim = F.hardtanh(sim, -1.0 + 1e-6, 1.0 - 1e-6)
    return torch.acos(sim) * (180 / np.pi)


class GazeAngularLoss(object):

    def __init__(self, key_true='gaze_a', key_pred='gaze_a_hat'):
        self.key_true = key_true
        self.key_pred = key_pred

    def __call__(self, gaze_a, gaze_hat):
        def pitchyaw_to_vector(pitchyaws):
            sin = torch.sin(pitchyaws)
            cos = torch.cos(pitchyaws)
            return torch.stack([cos[:, 0] * sin[:, 1], sin[:, 0], cos[:, 0] * cos[:, 1]], 1)
        y = pitchyaw_to_vector(gaze_a).detach()
        y_hat = gaze_hat
        if y_hat.shape[1] == 2:
            y_hat = pitchyaw_to_vector(y_hat)
        return torch.mean(nn_angular_distance(y, y_hat))
def mean_angle_loss(pred, truth):
    '''
    :param pred,truth: type=torch.Tensor
    :return:
    '''
    pred = pred.detach().numpy()
    ans = 0
    for i in range(len(pred)):
        p_x, p_y, p_z = (pred[i][j] for j in range(3))
        t_x, t_y, t_z = (truth[i][j] for j in range(3))
        # print("p_x={}, p_y={}, p_z={}".format(p_x, p_y, p_z))
        # print("t_x={}, t_y={}, t_z={}".format(t_x, t_y, t_z))
        angles = (p_x * t_x + p_y * t_y + p_z * t_z)/(math.sqrt(p_x**2+p_y**2+p_z**2) * math.sqrt(t_x**2+t_y**2+t_z**2))
        ans += math.acos(angles) * 180 / np.pi
    return ans / len(pred)


In [ ]:
use_cuda = torch.cuda.is_available()
# use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")
import time
def mtrain(train_iter, model, opt, loss_fn):
  model.train()
  total_loss=0
  overall_loss=0
  st = time.time()
  for b, batch in enumerate(train_iter):
    inp1 = batch[0].to(device)
    # inp2 = batch[1].to(device)
    out = batch[2].to(device)
    # print(inp1.shape, inp2.shape, out.shape)
    optimizer.zero_grad()
    # output = model(inp2,inp1,out)
    output = model(inp1, out)  
    # print("hey")
    # print(output.shape)
    loss = loss_fn(out, output)
    loss.backward()
    optimizer.step()
    total_loss += loss.data.item()
    overall_loss += loss.data.item()
    if b % 100 == 0 and b != 0:
      total_loss = total_loss / 100
  return overall_loss/len(train_iter)
def evaluate(val_iter, model, loss_fn):
  with torch.no_grad():
    model.eval()
    total_loss = 0
    for b, batch in enumerate(val_iter):
      inp1 = batch[0].to(device)
      # inp2 = batch[1].to(device)
      out = batch[2].to(device)
      # output = model(inp2, inp1, out)
      output = model(inp1, out)

      loss = loss_fn(out, output)
      total_loss += loss.data.item()
    return total_loss/len(val_iter)

def eval(iter, loss_fn):
  with torch.no_grad():
    # model.eval()
    total_loss = 0
    for b, batch in enumerate(iter):
      out = batch[2].to(device)
      out1 = batch[3].to(device)
      loss = loss_fn(out, out1)
      total_loss += loss.data.item()
    return total_loss/len(iter)
  

In [ ]:
for seed in [0, 2, 12, 22, 32, 42]:
  print("---------------------------------------------------------------------------------------------------------------------------------------------------------")
  print("computing for seed", seed)
  best_v =[]
  best_ts = []
  face_gazes = []
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  np.random.seed(seed)
  random.seed(seed)
  torch.backends.cudnn.deterministic = True
  dic_p = [0 for _ in range(15)]
  #begin cross person training-evaluation loop
  for leave_out in range(15):
    start_time_person = time.time()
    data = []
    train = []
    test = []
    val = []
    cnt_test = datag[leave_out]["gaze"].shape[0]
    for i in range(cnt_test):
      test.append([datag[leave_out]["z_gaze"][i], datag[leave_out]["z_app"][i], datag[leave_out]["gaze"][i], datag[leave_out]["gazep"][i]])
    
    pq = 0
    for key in datag.keys():
      if(key==leave_out):
        continue
      else:
        cnt = datag[key]["gaze"].shape[0]
        pq += cnt
        for i in range(cnt):
          data.append([datag[key]["z_gaze"][i], datag[key]["z_app"][i], datag[key]["gaze"][i], datag[key]["gazep"][i]])
    np.random.shuffle(test)
    np.random.shuffle(data)
    print("Data Length", len(data))
    print("Test Length", len(test))

    frc = 0.1
    llo =  int(frc * len(data))
    train = data[:-llo]
    val = data[llo:]


    x1_train = []
    x1_val = []
    x1_test = []

    x2_train = []
    x2_val = []
    x2_test = []

    y_train = []
    y_val = []
    y_test = []

    yp_train = []
    yp_val = []
    yp_test = []



    for ls in train:
      x1_train.append(ls[0])
      x2_train.append(ls[1])
      y_train.append(ls[2])
      yp_train.append(ls[3])

    for ls in val:
      x1_val.append(ls[0])
      x2_val.append(ls[1])
      y_val.append(ls[2])
      yp_val.append(ls[3])

    for ls in test:
      x1_test.append(ls[0])
      x2_test.append(ls[1])
      y_test.append(ls[2])
      yp_test.append(ls[3])

    x1_train = torch.tensor(np.array(x1_train))
    x1_val = torch.tensor(np.array(x1_val))
    x1_test = torch.tensor(np.array(x1_test))

    x2_train = torch.tensor(np.array(x2_train))
    x2_val = torch.tensor(np.array(x2_val))
    x2_test = torch.tensor(np.array(x2_test))

    y_train = torch.tensor(np.array(y_train))
    y_val = torch.tensor(np.array(y_val))
    y_test = torch.tensor(np.array(y_test))

    yp_train = torch.tensor(np.array(yp_train))
    yp_val = torch.tensor(np.array(yp_val))
    yp_test = torch.tensor(np.array(yp_test))


    train = torch.utils.data.TensorDataset(x1_train,x2_train,y_train, yp_train)
    train_data = torch.utils.data.DataLoader(train, batch_size=32)

    dev = torch.utils.data.TensorDataset(x1_val,x2_val,y_val, yp_val)
    dev_data = torch.utils.data.DataLoader(dev, batch_size=32)

    test_ = torch.utils.data.TensorDataset(x1_test,x2_test,y_test, yp_test)
    test_data = torch.utils.data.DataLoader(test_, batch_size=32)


    
    gaze2d = Gazeonly(6).to(device)
    lr = 1e-1
    n_epochs = 100
    train_losses = []
    val_losses = []
    # loss_fn = torch.nn.SmoothL1Loss(reduction="mean")
    loss_fn2 = mean_angle_loss
    loss_fn = GazeAngularLoss()
    optimizer = optim.SGD(gaze2d.parameters(), lr=lr)
    best_val = 100
    best_epoch = -1
    best_test = 100
    patience = 2
    cnt_patience_break  = 0
    test_iter = iter(test_data)
    facegaze = eval(test_iter, loss_fn)
    sp = time.time()
    for epoch in range(n_epochs):
      print("epoch ", epoch)
      train_iter = iter(train_data)
      train_loss = mtrain(train_iter, gaze2d, optimizer, loss_fn)
      train_losses.append(train_loss)
      val_iter = iter(dev_data)
      val_loss = evaluate(val_iter, gaze2d, loss_fn)
      test_iter = iter(test_data)
      test_loss = evaluate(test_iter, gaze2d, loss_fn)
      val_losses.append(val_loss)
      # print(epoch, train_loss, val_loss, test_loss)
      print("Time for this epoch ", time.time()-sp, train_loss, val_loss) 
      sp = time.time()
      if(val_loss < best_val):
        best_val = val_loss
        best_epoch = epoch
        best_test = test_loss
      else:
        cnt_patience_break += 1
        if(cnt_patience_break>=patience):
          break
    print(leave_out, best_val, best_test, facegaze, cnt_test)
    dic_p[leave_out] += best_val
    best_v.append(best_val)
    best_ts.append(best_test) 
    face_gazes.append(facegaze)
    print(time.time()-start_time_person)
print(dic_p/6)